In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
# Retrieve Google API key from config.py
from config import gkey

In [2]:
data_one_path = 'Resources/airbnbmark1.csv'
data_two_path = 'Resources/AB_NYC_2019.csv'
data_one = pd.read_csv(data_one_path)
data_two = pd.read_csv(data_two_path)

C:\Users\Saidah\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_one.head(1)

,id,name,summary,host_id,host_response_rate,host_acceptance_rate,host_is_superhost,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,...,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",2845,50%,38%,f,Midtown,Midtown,Manhattan,...,365,48,94.0,9.0,9.0,10.0,10.0,10.0,9.0,0.39


In [4]:
ident = data_one['id']
name = data_one['name']
rpm = data_one['reviews_per_month']
peoples = data_one['accommodates']
cleaning = data_one['cleaning_fee']
avg_rating = data_one['review_scores_rating']
accuracy = data_one['review_scores_rating']
cleanliness = data_one['review_scores_rating']
checkin = data_one['review_scores_checkin']
comm = data_one['review_scores_communication']
location = data_one['review_scores_location']
value = data_one['review_scores_value']

basic_info_df = pd.DataFrame({'ID': ident,
                             'Room Name': name,
                              'Accomodates': peoples,
                              'Cleaning Fee': cleaning,
                             'Reviews Per Month': rpm,
                             'Rating Score': avg_rating,
                             'Accuracy Score': accuracy,
                             'Cleanliness': cleanliness,
                             'Checkin Score': checkin,
                             'Communication Score': comm,
                             'Location Score': location,
                             'Value Score': value})
basic_info_df.head(2)

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,Location Score,Value Score
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,10.0,9.0
1,3831,Cozy Entire Floor of Brownstone,3,NaN,4.69,90.0,90.0,90.0,10.0,9.0,10.0,9.0


In [5]:
ident2 = data_two['id']
host_id = data_two['host_id']
borough = data_two['neighbourhood_group']
neighborhood = data_two['neighbourhood']
lat = data_two['latitude']
lng = data_two['longitude']
room_type = data_two['room_type']
price = data_two['price']
review_num = data_two['number_of_reviews']
avail = data_two['availability_365']

In [6]:
basic_two_df = pd.DataFrame({'ID': ident2,
                            'Host ID': host_id,
                            'Borough': borough,
                            'Neighbourhood': neighborhood,
                            'Latitude': lat,
                            'Longitude': lng,
                            'Type Room': room_type,
                            'Price (per night)': price,
                            'Number of Reviews': review_num,
                            'Availability per 365': avail})
basic_two_df

,ID,Host ID,Borough,Neighbourhood,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,9,365
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,0,365
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,270,194
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,9,0
...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,0,9
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,0,36
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,0,27
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,0,2


In [7]:
combined_data = pd.merge(basic_info_df, basic_two_df, on = 'ID')
cleaned_data = combined_data.dropna(how = 'any')
cleaned_data

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,...,Value Score,Host ID,Borough,Neighbourhood,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
1,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
2,2595,Skylit Midtown Castle,2,$95.00,0.38,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
6,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,10.0,...,9.0,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129
7,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.60,90.0,90.0,90.0,10.0,10.0,...,9.0,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113091,35596905,CONEY ISLAND BEACH close to Manhattan New York,4,$100.00,0.28,100.0,100.0,100.0,10.0,10.0,...,10.0,29622548,Brooklyn,Coney Island,40.58101,-73.98514,Entire home/apt,149,0,297
113092,35706482,Trendy Williamsburg Apartment,2,$50.00,0.13,100.0,100.0,100.0,10.0,10.0,...,10.0,16342788,Brooklyn,Williamsburg,40.70690,-73.94521,Private room,79,0,342
113093,35948011,"Classic, Urban with a splash of Glam! Near M...",3,$119.00,2.42,97.0,97.0,97.0,10.0,10.0,...,10.0,269165235,Manhattan,East Village,40.72546,-73.98703,Entire home/apt,235,1,326
113094,36176586,BRAND NEW KING 2br Airports& Times Sq MINUTES ...,10,$80.00,2.50,85.0,85.0,85.0,9.0,8.0,...,9.0,272080566,Queens,Elmhurst,40.73480,-73.86906,Entire home/apt,199,0,157


In [8]:
# cleaned_data.to_csv('Resources/cleaned_data.csv')

In [9]:
#Find location for Manhattan, Brooklyn, and Bronx
boroughs_df = pd.read_csv("Resources/Boroughs.csv")
boroughs_df.head()


,Boroughs
0,The Bronx
1,Brooklyn
2,Manhattan


In [10]:
boroughs_df["borough lng"]= ""
boroughs_df["borough lat"]= ""
boroughs_df["attraction name"]= ""
boroughs_df["attraction lng"]= ""
boroughs_df["attraction lat"]= ""
boroughs_df

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,,,,,
1,Brooklyn,,,,,
2,Manhattan,,,,,


In [11]:
params = {"key": gkey}

for index, row in boroughs_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['Boroughs']
    state = 'New York'

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    boroughs_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
#     print(cities_lat_lng.url)
    
#     # convert to json
    boroughs_lat_lng = boroughs_lat_lng.json()

    boroughs_df.loc[index, "borough lat"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lat"]
    boroughs_df.loc[index, "borough lng"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lng"]

# # Visualize to confirm lat lng appear
boroughs_df.head()

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.8648,40.8448,,,
1,Brooklyn,-73.9442,40.6782,,,
2,Manhattan,-73.9712,40.7831,,,


In [12]:
#Get locations for 1 attraction in each borough
params = {
    "radius": 10000,
    "types": "tourist_attraction",
    "key": gkey
}

for index, row in boroughs_df.iterrows():
    # get lat, lng from df
    lat = row["borough lat"]
    lng = row["borough lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    attraction = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(attraction.url)

    # convert to json
    attraction = attraction.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        boroughs_df.loc[index, "attraction name"] = attraction["results"][0]["name"]
        boroughs_df.loc[index, "attraction lng"] = attraction["results"][0]["geometry"]["location"]["lng"]
        boroughs_df.loc[index, "attraction lat"] = attraction["results"][0]["geometry"]["location"]["lat"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
boroughs_df

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.8648,40.8448,Bronx Zoo,-73.877,40.8506
1,Brooklyn,-73.9442,40.6782,Brooklyn Children's Museum,-73.944,40.6745
2,Manhattan,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [14]:
boroughs_df = boroughs_df.rename(columns={'Boroughs':'Borough'})
boroughs_df

,Borough,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.8648,40.8448,Bronx Zoo,-73.877,40.8506
1,Brooklyn,-73.9442,40.6782,Brooklyn Children's Museum,-73.944,40.6745
2,Manhattan,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [16]:
boroughs_df =boroughs_df.set_index('Borough')
boroughs_df

,borough lng,borough lat,attraction name,attraction lng,attraction lat
Borough,,,,,
The Bronx,-73.8648,40.8448,Bronx Zoo,-73.877,40.8506
Brooklyn,-73.9442,40.6782,Brooklyn Children's Museum,-73.944,40.6745
Manhattan,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [17]:
boroughs_final = boroughs_df.rename(index={'The Bronx':'Bronx'})
boroughs_final

,borough lng,borough lat,attraction name,attraction lng,attraction lat
Borough,,,,,
Bronx,-73.8648,40.8448,Bronx Zoo,-73.877,40.8506
Brooklyn,-73.9442,40.6782,Brooklyn Children's Museum,-73.944,40.6745
Manhattan,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [18]:
boroughs_final = boroughs_final.reset_index()
boroughs_final

,Borough,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,Bronx,-73.8648,40.8448,Bronx Zoo,-73.877,40.8506
1,Brooklyn,-73.9442,40.6782,Brooklyn Children's Museum,-73.944,40.6745
2,Manhattan,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [20]:
bnb_attr_merge = pd.merge(cleaned_data, boroughs_final, on = "Borough", how = 'outer')
bnb_attr_merge.head()

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,...,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813
1,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813
2,2595,Skylit Midtown Castle,2,$95.00,0.38,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813
3,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,10.0,...,-73.97500,Entire home/apt,200,74,129,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813
4,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.60,90.0,90.0,90.0,10.0,10.0,...,-73.97500,Entire home/apt,200,74,129,-73.9712,40.7831,American Museum of Natural History,-73.974,40.7813


In [24]:
#drop columns that are not needed in the merged dataset
final_df = bnb_attr_merge.drop(columns= ['borough lng','borough lat'])
final_df

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,...,Neighbourhood,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365,attraction name,attraction lng,attraction lat
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355,American Museum of Natural History,-73.974,40.7813
1,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355,American Museum of Natural History,-73.974,40.7813
2,2595,Skylit Midtown Castle,2,$95.00,0.38,94.0,94.0,94.0,10.0,10.0,...,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355,American Museum of Natural History,-73.974,40.7813
3,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,10.0,...,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129,American Museum of Natural History,-73.974,40.7813
4,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.60,90.0,90.0,90.0,10.0,10.0,...,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129,American Museum of Natural History,-73.974,40.7813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78302,33509538,WALK TO FREE SI FERRY FROM FURNISHED STUDIO APT,2,$100.00,1.43,80.0,80.0,80.0,10.0,10.0,...,St. George,40.64015,-74.08127,Entire home/apt,99,4,326,NaN,NaN,NaN
78303,33914554,3 Bedroom Full house - 15mins to NYC/Ferry,10,$95.00,0.48,85.0,85.0,85.0,10.0,10.0,...,Todt Hill,40.61041,-74.11593,Entire home/apt,135,2,36,NaN,NaN,NaN
78304,35224304,Spacious Home Away from Home,10,$100.00,1.10,91.0,91.0,91.0,10.0,10.0,...,Grymes Hill,40.61569,-74.08957,Entire home/apt,125,1,44,NaN,NaN,NaN
78305,214917,New Clean Spacious Bed & Breakfast,3,$15.00,0.04,100.0,100.0,100.0,8.0,8.0,...,Emerson Hill,40.60742,-74.14388,Private room,80,2,158,NaN,NaN,NaN
